In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 参数管理
参数管理的内容包含: 
- 访问参数, 用于调试、诊断和可视化
- 参数初始化
- 在不同模型组件之间共享参数

In [3]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0991],
        [0.0864]], grad_fn=<AddmmBackward0>)

## 参数访问
可以访问已有模型的参数, 使用 `nn.Sequential`定义模型的时候, 可以通过索引来访问模型的任意层

In [4]:
# 访问一个模型的全部参数
print(net[0].state_dict())

OrderedDict([('weight', tensor([[-0.1140,  0.2452, -0.3135, -0.0166],
        [-0.0059, -0.3440,  0.1467, -0.0842],
        [ 0.1440, -0.1320,  0.4097,  0.0152],
        [ 0.0424, -0.4933, -0.2836, -0.3300],
        [ 0.2844, -0.2255, -0.0505,  0.1970],
        [ 0.4932,  0.4995,  0.3552, -0.1307],
        [ 0.3465,  0.0710,  0.0840, -0.1203],
        [-0.2880, -0.4392, -0.4636, -0.3366]])), ('bias', tensor([ 0.3584,  0.0277, -0.4997,  0.1030,  0.0103,  0.3437, -0.1254, -0.3330]))])


### 目标参数
每一个参数都表示为一个参数类(`nn.parameter.Parameter`)的一个实例

In [7]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2667], requires_grad=True)
tensor([0.2667])


In [8]:
# 此时没有使用反向传播, 所以相关参数的梯度为 None
net[2].weight.grad == None, net[2].bias.grad == None

(True, True)

### 一次性访问所有参数
当需要对所有参数执行操作的时候, 逐个访问参数可能比较麻烦, 处理更加复杂的块的时候, 情况会更加复杂, 此时需要遍历整个树来提取每一个子树的参数

In [9]:
# 1. 访问第一层的所有参数
# 注意 net[0] 和 net 都是 nn.Module 类型
# 注意其中的 * 表示分散列表中的元素类似于 ...
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
# 2. 访问网络中的所有参数
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [10]:
# 3. 利用索引访问
net.state_dict()['2.bias']

tensor([0.2667])

### 从嵌套块收集参数
可以使用 `nn.Module` 对象的 `add_module` 方法来添加重复的块, 从而完全块的嵌套

In [11]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3080],
        [-0.3080]], grad_fn=<AddmmBackward0>)

In [12]:
# 网络结构如下
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [14]:
# 返回第一个块, 第二个子块的第一层
rgnet[0][1][0].bias.data

tensor([ 0.4021, -0.4630,  0.3853, -0.4227,  0.0936, -0.0953, -0.1855,  0.2166])

## 参数初始化
`PyTorch`中提供默认随机初始化, 也允许我们创建自定义初始化的方法, 默认情况下 `PyTorch`会更具一个范围均匀地初始化权重和偏置矩阵, 这一个范围是根据输入和输出维度计算的, `PyTorch` 中的 `nn.init` 模块提供了多种预置初始化方法

### 内置初始化
使用 `nn.init` 模块中的各种方法, 比如 `nn.init.normal_` 或者 `nn.init.zeros_` 可可以初始化对应的参数, 结合 `net.apply(fn)` 方法, 这一个方法可以把 `fn` 作用在 `net` 的所有子模块中

In [17]:
# 高斯分布: normal_, 置0: zeros_
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0134, -0.0050, -0.0047,  0.0031]), tensor(0.))

In [23]:
# 初始化为常数 constant_
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data,net[2].weight.data

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
 tensor([[1., 1., 1., 1., 1., 1., 1., 1.]]))

In [25]:
# 使用 xavier 初始化方法进行初始化, 主要是 nn.Module 对象就可以初始化
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[ 0.0261,  0.5972,  0.0540, -0.1318],
        [ 0.0822,  0.0775,  0.4442, -0.0087],
        [ 0.5582, -0.1029, -0.6108,  0.3070],
        [-0.5713, -0.4307,  0.6260,  0.6819],
        [ 0.3809,  0.2615, -0.5368, -0.1770],
        [ 0.6146, -0.3754,  0.5280,  0.4743],
        [-0.6857,  0.3639, -0.6416, -0.1110],
        [ 0.3743, -0.4182,  0.2518, -0.2293]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化
使用一样的初始化方法 + `net.apply` 即可, 假设利用如下初始化方法:
$$
w \sim 
\begin{cases} 
U(5, 10) & \text{可能性 } \frac{1}{4} \\
0 & \text{可能性 } \frac{1}{2} \\
U(-10, -5) & \text{可能性 } \frac{1}{4} 
\end{cases}
$$

In [29]:
def my_init(m):
    if type(m) == nn.Linear:
        nn.init.uniform_(m.weight, -10, 10)
        # 类似于 bool 型索引, 使用矩阵代替元素
        m.weight.data *= (m.weight.data.abs() >= 5)
net.apply(my_init)
net[0].weight[:2]

tensor([[ 6.5319,  5.5679,  9.8325, -5.2827],
        [-0.0000,  5.1967, -6.1572,  0.0000]], grad_fn=<SliceBackward0>)

In [30]:
# 直接设置参数
net[0].weight.data[:] = 100
net[0].weight.data[0, 0] = 1000
net[0].weight.data

tensor([[1000.,  100.,  100.,  100.],
        [ 100.,  100.,  100.,  100.],
        [ 100.,  100.,  100.,  100.],
        [ 100.,  100.,  100.,  100.],
        [ 100.,  100.,  100.,  100.],
        [ 100.,  100.,  100.,  100.],
        [ 100.,  100.,  100.,  100.],
        [ 100.,  100.,  100.,  100.]])

## 参数绑定
为了在多个层之间共享参数, 只需要定义一个共享层, 之后利用它的参数来设置另一个层的参数

In [33]:
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                   shared, nn.ReLU(),
                   shared, nn.ReLU(),
                   nn.Linear(8, 1))
net(X)
# 使用列表/ndarray/tensor 代替元素
print(net[2].weight.data == net[4].weight.data) # 注意 bool 型数组/索引的使用

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])
